In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from langchain import hub
from langchain_community.vectorstores import Chroma
import pickle

Document loader

In [2]:
pages = PyPDFLoader("src/Constitution_de_la_republique_tunisiennefr.pdf").load_and_split()
len(pages)

19

Splitter


In [3]:
doc_chunks = []
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    separators=["\nArticle", "\n\n", "\n", ".", "!", "?", ",", " ", ""],
    chunk_overlap=50,
)
chunks = text_splitter.split_documents(pages)
# for i, chunk in enumerate(chunks):
#     doc = Document(
#         page_content=chunk, metadata={"page": doc.metadata["page"], "chunk": i}
#     )
#     doc.metadata["source"] = f"{doc.metadata['page']}-{doc.metadata['chunk']}"
#     doc.metadata["filename"] = "Constitution_de_la_republique_tunisienne" 
#     doc_chunks.append(doc)
len(chunks)

142

Vectorstores

In [4]:
# model =  SentenceTransformer("dangvantuan/sentence-camembert-large")

# sentences = ["Un avion est en train de décoller.",
#           "Un homme joue d'une grande flûte.",
#           "Un homme étale du fromage râpé sur une pizza.",
#           "Une personne jette un chat au plafond.",
#           "Une personne est en train de plier un morceau de papier.",
#           ]

# embeddings = model.encode(sentences)

In [5]:
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/dangvantuan/sentence-camembert-large")

In [6]:
!pip install chromadb

  Using cached chromadb-0.4.24-py3-none-any.whl.metadata (7.3 kB)
  Using cached build-1.2.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached chroma-hnswlib-0.7.3.tar.gz (31 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached fastapi-0.110.1-py3-none-any.whl.metadata (24 kB)
  Using cached uvicorn-0.29.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached posthog-3.5.0-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached pulsar_client-3.4.0-cp312-cp312-win_amd64.whl.metadata (1.0 kB)
  Using cached onnxruntime-1.17.1-cp312-cp312-win_amd64.whl.metadata (4.4 kB)
  Using cached opentelemetry_api-1.24.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.24.0-

  error: subprocess-exited-with-error
  
  × Building wheel for chroma-hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'hnswlib' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib
ERROR: Could not build wheels for chroma-hnswlib, which is required to install pyproject.toml-based projects


In [7]:
embeddings = HuggingFaceHubEmbeddings(model="dangvantuan/sentence-camembert-large")
index = Chroma.from_documents(chunks, embeddings)

c:\Users\user\.pyenv\pyenv-win\versions\3.12.1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.

In [ ]:
index.save_local("faiss_index")
# loaded_index = FAISS.load_local("faiss_index", embeddings)

Retrieval

In [ ]:
retriever = index.as_retriever(search_kwargs={"k": 3})
retriever.invoke("quelle est la réligion de la tunisie?")

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")

llm.invoke("Tell me a joke")

"\nWhy don't scientists trust atoms? Because they make up everything! 😂"

In [ ]:
query = "Tell me a joke"

for chunks in llm.stream(query):
    print(chunks)

S
ure
!
 Here
'
s
 one
:




Why
 don
'
t
 scient
ists
 trust
 atoms
?


B
ecause
 they
 make
 up
 everything
!




I
 hope
 that
 brought
 a
 smile
 to
 your
 face
!
 Do
 you
 have
 any
 specific
 topic
 or
 theme
 in
 mind
 for
 a
 jo
ke
,
 and
 I
 can
 try
 to
 come
 up
 with
 something
 tail
ored
 to
 your
 prefer
ences
?

